### Question Two

We are asked to solve an issue for a car rental agency. There are 10 locations, and each one has a certain number of cars on site, but also a certain number that they need. We are given a cost of moving a car from one site to another ($0.50 per mile). Now, we are to set up a linear program such that we can solve this issue. We denote each agency as ":ag1", and so forth. 

The answer is as follows: 

In [1]:
using JuMP, Clp, NamedArrays

m = Model(solver=ClpSolver())

# cost of moving
cost = 0.50;

# demand and stock of cars at each location
demand = [10,   6,   8,  11,   9,  7,  15,   7,   9,  12];
stock  = [ 8 , 13 ,  4  , 8  ,12  , 2 , 14 , 11 , 15 ,  7];

# coordinate pair of where each location is at
xcord  = [ 0,  20,  18,  30  ,35 , 33 ,  5 ,  5 , 11 ,  2];
ycord  = [ 0  ,20,  10,  12  , 0  ,25 , 27,  10 ,  0  ,15];

agency = [ :ag1, :ag2, :ag3, :ag4, :ag5, :ag6, :ag7, :ag8, :ag9, :ag10]


# calculate the distance from one location to the next
distance = zeros(10,10)
for i = 1:10
    for j = 1:10
        distance[i,j] = 1.3(sqrt( (xcord[i]-(xcord[j]) )^2 + (ycord[i]-(ycord[j]) )^2 ) )
    end
end

@variable(m, x[1:10, 1:10] >= 0)

@constraint(m, sup[i in 1:10], sum(x[i,j] for j in 1:10) == stock[i] )   # supply constraint
@constraint(m, dem[j in 1:10], sum(x[i,j] for i in 1:10) == demand[j] )   # demand constraint

#objective : minimize cost of moving vehicles --> Idea here : i think we need a flow constraint (sailco)
@objective(m, Min, 0.5*sum( x[i,j]*distance[i,j] for i in 1:10, j in 1:10 ) ) 

status = solve(m)
print(status)
println()
println()

solution = NamedArray( Int[getvalue(x[i,j]) for i in 1:10, j in 1:10], (agency,agency), ("demand","stock") )
println( solution )
println()
println("Total cost will be \$", getobjectivevalue(m))

Optimal

10×10 Named Array{Int64,2}
demand ╲ stock │  :ag1   :ag2   :ag3   :ag4  …   :ag7   :ag8   :ag9  :ag10
───────────────┼──────────────────────────────────────────────────────────
:ag1           │     8      0      0      0  …      0      0      0      0
:ag2           │     0      6      1      0         1      0      0      0
:ag3           │     0      0      4      0         0      0      0      0
:ag4           │     0      0      0      8         0      0      0      0
:ag5           │     0      0      0      3         0      0      0      0
:ag6           │     0      0      0      0         0      0      0      0
:ag7           │     0      0      0      0        14      0      0      0
:ag8           │     0      0      0      0         0      6      0      5
:ag9           │     2      0      3      0         0      1      9      0
:ag10          │     0      0      0      0  …      0      0      0      7

Total cost will be $152.63901632295628
